# Filtering Fake Reviews

### Problem Statement
As most of the people require review about a product before spending their money on the product. So people come across various reviews in the website but these reviews are genuine or fake is not identified by the user. In some review websites some good reviews are added by the product company people itself in order to make product famous this people belong to Social Media Optimization team. They give good reviews for many different products manufactured by their own firm. User will not be able to find out whether the review is genuine or fake.

In [37]:
import pandas as pd
import random
import re
import pickle
import nltk 

In [38]:
dataset = pd.read_csv("reviews.csv",sep="\t")
# dataset.drop(columns= "Unnamed: 0",inplace = True)
dataset.drop(dataset.columns[dataset.columns.str.contains('unnamed: 0',case = False)],axis = 1)
dataset.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,timestamp,IP Address
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,82850235,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,1.440988e+09,193.93.167.87
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,82850235,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0.0,1.0,N,Y,five star,it's great,2015-08-31,1.441002e+09,193.93.167.87
2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0.0,0.0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31,1.440959e+09,193.93.167.87
3,US,4332923,RRB9C05HDOD4O,B00QUFTPV4,82850235,abcGoodefg® FBI Covert Acoustic Tube Earpiece ...,Mobile_Electronics,4,0.0,0.0,N,Y,Work very well,Work very well,2015-08-31,1.441015e+09,193.93.167.87
4,US,44855305,R26I2RI1GFV8QG,B0067XVNTG,563475445,Generic Car Dashboard Video Camera Vehicle Vid...,Mobile_Electronics,2,0.0,0.0,N,Y,Cameras has battery issues,"Be careful with these products, I have bought ...",2015-08-31,1.440973e+09,205.10.168.66


In [39]:
import re
import collections

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import nltk
from nltk.corpus import stopwords 
from nltk.corpus import wordnet

In [40]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/eashan22/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/eashan22/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [41]:
dataset.set_index("review_id",inplace=True)
dataset

,marketplace,customer_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,timestamp,IP Address
review_id,,,,,,,,,,,,,,,,
R8MEA6IGAHO0B,US,20422322,B00MC4CED8,82850235,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,1.440988e+09,193.93.167.87
R31LOQ8JGLPRLK,US,40835037,B00OQMFG1Q,82850235,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0.0,1.0,N,Y,five star,it's great,2015-08-31,1.441002e+09,193.93.167.87
R2Y0MM9YE6OP3P,US,51469641,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0.0,0.0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31,1.440959e+09,193.93.167.87
RRB9C05HDOD4O,US,4332923,B00QUFTPV4,82850235,abcGoodefg® FBI Covert Acoustic Tube Earpiece ...,Mobile_Electronics,4,0.0,0.0,N,Y,Work very well,Work very well,2015-08-31,1.441015e+09,193.93.167.87
R26I2RI1GFV8QG,US,44855305,B0067XVNTG,563475445,Generic Car Dashboard Video Camera Vehicle Vid...,Mobile_Electronics,2,0.0,0.0,N,Y,Cameras has battery issues,"Be careful with these products, I have bought ...",2015-08-31,1.440973e+09,205.10.168.66
RY8DDL22YG4R5,US,7846966,B00KA6CCVY,563475445,Aweek® Air Acoustic Earpiece Headset for Baofe...,Mobile_Electronics,3,0.0,1.0,N,Y,"It appears to be good, but I'm still waiting f...","It appears to be good, but I'm still waiting f...",2015-08-31,1.440978e+09,205.10.168.66
R2AT2426ZHFUHH,US,21299354,B00MJCDPM2,563475445,Sentey LS-4460 B-Trek S8 Bluetooth Portable St...,Mobile_Electronics,3,0.0,1.0,N,Y,Didn't love the first one,"First one arrived as a brick. Wouldn't work, ...",2015-08-31,1.441033e+09,205.10.168.66
R8MEA6IGAHO0B,US,20422322,B00MC4CED8,563475445,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,1.441013e+09,205.10.168.66
R8MEA6IGAHO0B,US,20422322,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,1.440988e+09,206.33.134.174


In [42]:
# Latent symantic analysis
# it will analyse all reviews and determine all reviews belong to the same concept
def LSA(text):
    #text is list of reviews of same product
    
    
    # Created TF-IDF Model
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(text)
    
    # Created SVD(Singular Value Decomposition)
    lsa = TruncatedSVD(n_components = 1,n_iter = 100)
    lsa.fit(X)
    
    terms = vectorizer.get_feature_names()
    concept_words={}

    for j,comp in enumerate(lsa.components_):
        componentTerms = zip(terms,comp)
        sortedTerms = sorted(componentTerms,key=lambda x:x[1],reverse=True)
        sortedTerms = sortedTerms[:10]
        concept_words[str(j)] = sortedTerms
     
    sentence_scores = []
    for key in concept_words.keys():
        for sentence in text:
            words = nltk.word_tokenize(sentence)
            scores = 0
            for word in words:
                for word_with_scores in concept_words[key]:
                    if word == word_with_scores[0]:
                        scores += word_with_scores[1]
            sentence_scores.append(scores)
    return sentence_scores

In [43]:
product_df = dataset.groupby("product_id")
#grouping dataset by product_id

unique_products = dataset["product_id"].unique()
#stores list of all product id

no_products = len(unique_products.tolist())
#no. of products

remove_reviews = []
#store review_id that are fake 
    
for i in range(no_products):
    #iterate through all product reviews 
    
    df = product_df.get_group(unique_products[i])
    #dataframe of a single product
    
    unique_reviews = df.index.tolist()
    #list of review_id of reviews of same product
    
    no_reviews = len(unique_reviews)   
    #no. of reviews of same product
    
    count = no_reviews 
    #count is no. of reviews that can be analysed
    
    reviews = []
    #list of review texts
    
    review_id = []
    #list of review texts
    
    for j in range(no_reviews):
        #iterate through all reviews
        
        text = str(df.loc[unique_reviews[j]]["review_body"])
        # text of a review 
        
        #cleaning the text
        text = re.sub(r"\W"," ",text)             
        text = re.sub(r"\d"," ",text)             
        text = re.sub(r"\s+[a-z]\s+"," ",text)    
        text = re.sub(r"^[a-z]\s+"," ",text)    
        text = re.sub(r"\s+[a-z]$"," ",text)    
        text = re.sub(r"\s+"," ",text)
        
        words = nltk.word_tokenize(text)
        #text into word list
        
        if(len(words)==1):
        #if only one word in text review
            
            if(len(text) <=1 or not wordnet.synsets(text) ):
            #if word is having only one character or invalid english word
                
                remove_reviews.append(unique_reviews[j])
                #append this review as fake
                
                count-=1
                #review left to be analysed will be decrease by 1
                
                continue
                #check for next review
                '''
            elif(len(words[0])<=1):
                remove_reviews.append(unique_reviews[j])
                count-=1
                continue
                '''
        elif(len(words)==0):
        #if no words
            
            remove_reviews.append(unique_reviews[j])
            #append this review as fake
            
            count-=1
            #review left to be analysed will be decrease by 1
            
            continue
            #check for next review
        
        review_id.append(unique_reviews[j])
        #valid: append review_id to review_id list for analysis
        
        reviews.append(text)
        #valid: append review_body to reviews list for analysis
        
    ###########################################################################################
    if(count<=0):
        #if no reviews left to analyse 

        continue
        #check for next
        
    if(count==1): 
        #if only one review is left to analyse
        
        #check concept between product title and the review
        
        text = [text,str(df.loc[review_id[0]]["product_title"])] 
        #add product_title and review to the list 
        
        sentence_scores = LSA(text) 
        #call LSA method to get the score
        
        if(sentence_scores[0]==0): 
        #if review score is 0, then it's fake
            remove_reviews.append(review_id[0])
        continue
    
    #list of scores of reviews of same product
    sentence_scores = LSA(reviews)
            
    for j in range(len(sentence_scores)):
        #iterating through all the scores
        
        if(sentence_scores[j]==0.00):
            # if score is 0, it's fake
            remove_reviews.append(review_id[j])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/truncated_svd.py:191: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


In [45]:
len(remove_reviews)

6604